In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import seaborn as sns

from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data_raw = pd.read_csv(filename)
training_data_raw


,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [17]:
training_data_raw['Well Name'] = training_data_raw['Well Name'].astype('category')
training_data_raw['Formation'] = training_data_raw['Formation'].astype('category')
training_data_raw['Well Name'].unique()

[SHRIMPLIN, ALEXANDER D, SHANKLE, LUKE G U, KIMZEY A, CROSS H CATTLE, NOLAN, Recruit F9, NEWBY, CHURCHMAN BIBLE]
Categories (10, object): [SHRIMPLIN, ALEXANDER D, SHANKLE, LUKE G U, ..., NOLAN, Recruit F9, NEWBY, CHURCHMAN BIBLE]

In [19]:
# 1=sandstone  2=c_siltstone   3=f_siltstone 
# 4=marine_silt_shale 5=mudstone 6=wackestone 7=dolomite
# 8=packstone 9=bafflestone
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS',
                 'WS', 'D','PS', 'BS']
#facies_color_map is a dictionary that maps facies labels
#to their respective colors
facies_color_map = {}
for ind, label in enumerate(facies_labels):
    facies_color_map[label] = facies_colors[ind]

def label_facies(row, labels):
    return labels[ row['Facies'] -1]
    
training_data_raw.loc[:,'FaciesLabels'] = training_data.apply(lambda row: label_facies(row, facies_labels), axis=1)
training_data_raw.replace(to_replace=np.nan, value=-99999.0, inplace=True)
training_data_raw.describe()


,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000
mean,4.503254,2906.867438,64.933985,0.659566,4.402484,13.201066,-22098.588517,1.518438,0.521852
std,2.474324,133.300164,30.302530,0.252703,5.274947,7.132846,41499.330187,0.499720,0.286644
min,1.000000,2573.500000,10.149000,-0.025949,-21.832000,0.550000,-99999.000000,1.000000,0.000000
25%,2.000000,2821.500000,44.730000,0.498000,1.600000,8.500000,2.416000,1.000000,0.277000
50%,4.000000,2932.500000,64.990000,0.639000,4.300000,12.020000,3.300000,2.000000,0.528000
75%,6.000000,3007.000000,79.438000,0.822000,7.500000,16.050000,4.000000,2.000000,0.769000
max,9.000000,3138.000000,361.150000,1.800000,19.312000,84.400000,8.094000,2.000000,1.000000


### Loading and adding the variables from the wavelet decomposition

In [22]:
dwt_vars = pd.read_pickle('data/vars_from_dwt.pkl')
dwt_vars.replace(to_replace=np.nan, value=-99999.0, inplace=True)
dwt_vars.shape

(4149, 32)

In [26]:
training_data = pd.merge(training_data_raw,dwt_vars,on=['Depth','Well Name'])
training_data.drop_duplicates(inplace=True)

In [27]:
training_data.shape

(4148, 42)

### Data exploration

In [4]:
# training_data.info()
# print('There are {0:2.1f}% NaN in the photoelectric variable'.format(training_data['PE'].isnull().sum()/len(training_data['PE'])*100))

There are 22.1% NaN in the photoelectric variable


One solution to deal with NaN will be to dropp rows where there is NaN in PE variables (as Brandon Hall made in his tutorial). 
However, as 22.1% of data is a pretty good amount of data. Scikit-learn offers a builtin function to replace NaN with mean, median or most-frequent value. 
For this excercise we propose we prefer to replace NaN with a "clearly out-of-range" value ( like -99999). This allow ensemble classifier, that are based on decision-trees, to split on the criteria "known values vs unknown values". 

In [67]:
# training_data['PE'].fillna(-99999, inplace=True)
# training_data

# PE_mask = training_data['PE'].notnull().values
# training_data = training_data[PE_mask]
# training_data.describe()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


Feauturize using cesium

## Test set
Remove a single well to use as a blind test later.

In [28]:
blind = training_data[training_data['Well Name'] == 'NEWBY']
training_data = training_data[training_data['Well Name'] != 'NEWBY']
training_data['Well Name'].unique()
blind

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,...,PHIND_cD_step_level_3,PHIND_cD_step_level_4,PHIND_cD_step_level_5,PHIND_cD_step_level_6,PE_cD_step_level_1,PE_cD_step_level_2,PE_cD_step_level_3,PE_cD_step_level_4,PE_cD_step_level_5,PE_cD_step_level_6
3288,3,A1 SH,NEWBY,2826.0,76.34,0.719,7.8,11.00,3.7,1,...,-0.638453,-0.638453,-0.638453,-0.638453,0.104104,0.104104,0.104104,0.104104,0.104104,0.104104
3289,3,A1 SH,NEWBY,2826.5,83.74,0.688,9.7,12.55,3.4,1,...,0.058437,0.058437,0.058437,0.058437,-0.014871,-0.014871,-0.014871,-0.014871,-0.014871,-0.014871
3290,3,A1 SH,NEWBY,2827.0,83.19,0.664,10.1,11.95,3.4,1,...,0.058437,0.058437,0.058437,0.058437,-0.014871,-0.014871,-0.014871,-0.014871,-0.014871,-0.014871
3291,3,A1 SH,NEWBY,2827.5,80.44,0.648,10.1,11.15,3.4,1,...,0.098691,0.098691,0.098691,0.098691,-0.036854,-0.036854,-0.036854,-0.036854,-0.036854,-0.036854
3292,3,A1 SH,NEWBY,2828.0,75.42,0.648,9.3,11.45,3.3,1,...,0.098691,0.098691,0.098691,0.098691,-0.036854,-0.036854,-0.036854,-0.036854,-0.036854,-0.036854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,6,C LM,NEWBY,3055.0,66.94,0.838,4.0,8.00,4.2,2,...,1.276647,1.276647,1.276647,1.276647,-0.114737,-0.114737,-0.114737,-0.114737,-0.114737,-0.114737
3747,6,C LM,NEWBY,3055.5,54.06,0.823,1.9,5.45,4.3,2,...,1.276647,1.276647,1.276647,1.276647,-0.114737,-0.114737,-0.114737,-0.114737,-0.114737,-0.114737
3748,6,C LM,NEWBY,3056.0,47.87,0.797,0.7,4.85,4.4,2,...,-1.397988,-1.397988,-1.397988,-1.397988,0.029744,0.029744,0.029744,0.029744,0.029744,0.029744
3749,6,C LM,NEWBY,3056.5,49.34,0.763,2.3,4.85,4.1,2,...,-1.397988,-1.397988,-1.397988,-1.397988,0.029744,0.029744,0.029744,0.029744,0.029744,0.029744


## Split into training and test set

Now we extract just the feature variables we need to perform the classification.  The predictor variables are the five wireline values and two geologic constraining variables. We also get a vector of the facies labels that correspond to each feature vector.

In [29]:
X_train = training_data.drop(['Formation', 'Well Name', 'Depth','Facies','FaciesLabels'], axis=1).values
y_train = training_data['Facies'].values
X_test = blind.drop(['Formation', 'Well Name', 'Depth','Facies','FaciesLabels'], axis=1).values
y_test = blind['Facies'].values

X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# feature_vectors = training_data.drop(['Formation', 'Well Name', 'Depth','Facies','FaciesLabels'], axis=1)
# feature_vectors.describe()

(3685, 37)
(463, 37)
(3685,)
(463,)


In [30]:
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

from sklearn.metrics import confusion_matrix
from classification_utilities import display_cm, display_adj_cm
from sklearn.metrics import classification_report

def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

adjacent_facies = np.array([[1], [0,2], [1], [4], [3,5], [4,6,7], [5,7], [5,6,8], [6,7]])

def accuracy_adjacent(conf, adjacent_facies):
    nb_classes = conf.shape[0]
    total_correct = 0.
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
        for j in adjacent_facies[i]:
            total_correct += conf[i][j]
    return total_correct / sum(sum(conf))

## Comparing classifier

In [37]:
classifiers = [
    ensemble.RandomForestClassifier(n_estimators=500),
    ensemble.GradientBoostingClassifier(n_estimators=50),
    ensemble.BaggingClassifier()]

The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator. Two families of ensemble methods are usually distinguished:
In averaging methods, the driving principle is to build several estimators independently and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced. Examples: Bagging methods, Forests of randomized trees, ...
By contrast, in boosting methods, base estimators are built sequentially and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble. Examples: AdaBoost, Gradient Tree Boosting, ...

In [38]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Precision", "Recall", "F1-score"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****\n')
    predicted_labels = clf.predict(X_test)
    
#     accuracy = metrics.accuracy_score(y_test, train_predictions)
#     print("Accuracy: {:.4%}".format(accuracy))
    conf = confusion_matrix(y_test, predicted_labels)

    print('Optimized facies classification accuracy = %.2f' % accuracy(conf))
    print('Optimized adjacent facies classification accuracy = %.2f\n' % accuracy_adjacent(conf, adjacent_facies))
    
    display_adj_cm(conf, facies_labels, adjacent_facies,display_metrics=True, hide_zeros=True)
    
    
    

#     precision = metrics.precision_score(y_test, train_predictions, average='micro')
#     print("Precision: {:.4%}".format(precision))
    
#     recall = metrics.recall_score(y_test, train_predictions, average='micro')
#     print("Recall: {:.4%}".format(recall))
    
#     f1 = metrics.f1_score(y_test, train_predictions, average='micro')
#     print("F1 score: {:.4%}".format(f1))
       
#     log_entry = pd.DataFrame([[name, accuracy*100, precision*100, recall*100, f1*100]], columns=log_cols)
#     log = log.append(log_entry)
    
print("="*30)

RandomForestClassifier
****Results****

Optimized facies classification accuracy = 0.51
Optimized adjacent facies classification accuracy = 0.90

     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS                                                           0
     CSiS          98                                              98
     FSiS     3          77                                        80
     SiSh                      41          13           4          58
       MS                 3          12           2    11          28
       WS                       1          94                 1    96
        D                                        16                16
       PS                             3                53          56
       BS                                   4                27    31

Precision  0.00  1.00  0.96  0.98  0.80  0.85  0.89  0.78  0.96  0.91
   Recall  0.00  1.00  0.96  0.71  0.43  0.98  1.00  0.95  0.87  0.90
   

In [ ]:
# from sklearn.metrics import confusion_matrix
# from classification_utilities import display_cm, display_adj_cm

# conf = confusion_matrix(y_test, train_predictions)
# display_cm(conf, facies_labels, hide_zeros=True)
display_adj_cm(conf, facies_labels, adjacent_facies, 
           display_metrics=True, hide_zeros=True)

In [ ]:
# conf = confusion_matrix(y_test, train_predictions)
# display_cm(conf, facies_labels, hide_zeros=True)
conf

In [ ]:
ax = sns.stripplot(x="Well Name", y="Facies", hue='Facies', data=training_data, jitter=False,palette="Set2")

In [ ]:
training_data['Facies']

In [ ]:
mask = training_data['PE'].isnull().values
training_data_test = training_data[mask]

In [ ]:
training_data_test['Well Name'].unique()

In [ ]:
def make_facies_log_plot(logs, facies_colors):
#     plt.style.use('seaborn-whitegrid')

    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')
    
    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster=np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=6, figsize=(8, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im=ax[5].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    
    divider = make_axes_locatable(ax[5])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((17*' ').join([' SS ', 'CSiS', 'FSiS', 
                                'SiSh', ' MS ', ' WS ', ' D  ', 
                                ' PS ', ' BS ']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([])
    ax[5].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

In [ ]:
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

In [ ]:
make_facies_log_plot(
    training_data[training_data['Well Name']=='Recruit F9'],
    facies_colors)